# Code to generate the plots from SynthPop Paper 1, Section 7.1

Notebook author: Macy Huston

Building on work by: Jonas Klüter

### Imports

In [ ]:
import numpy
import matplotlib.pyplot as plt
from matplotlib import gridspec
import sys
import numpy as np
import pandas
import pandas as pd

sys.path.append('..')
from synthpop_main import SynthPop
from synthpop_utils import half_cone_angle_to_solidangle
from synthpop_utils.coordinates_transformation import CoordTrans

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from astropy.coordinates import SkyCoord, Galactic
import astropy.units as u
from astroquery.gaia import Gaia, GaiaClass
from astroquery.vizier import Vizier

GaiaModel = GaiaClass()
GaiaModel.MAIN_GAIA_TABLE = "gaiadr3.gaia_universe_model"
GaiaModel.ROW_LIMIT = 10000000
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.ROW_LIMIT = 10000000

In [ ]:
# Set up bulge and far disk/halo test fields
locs=[[2,-2],[30,30]]
radii=[0.05*u.degree, 1*u.degree]
solangs = [radius**2*np.pi for radius in radii]

### SynthPop catalog generation - don't rerun location processing if not needed

In [ ]:
mod1 = SynthPop(sun={"x": 8, "y": 0, "z":0.015, "u":12.75, "v":231.53, "w":7.10},
                lsr={"u_lsr":0.0, "v_lsr":230.6, "w_lsr":0.0},
                name_for_output="gums", 
                model_name="GUMS_dr3", max_distance=25,
                extinction_map_kwargs={"name":"gums"}, 
                chosen_bands=["Gaia_G_EDR3", "Gaia_BP_EDR3", "Gaia_RP_EDR3"],
                maglim=["Gaia_G_EDR3", 21, "keep"], lost_mass_option=2,
                post_processing_kwargs={"name": "ProcessDarkCompactObjects"},
                output_location="/u/mhuston/code/synthpop/outputfiles/gums_comp",
                output_file_type="hdf5", overwrite=True, chunk_size=250000,
                evolution_class={"name":"MIST", "interpolator":"CharonInterpolator"}
               )
mod2 = SynthPop(sun={"x": 8, "y": 0, "z":0.015, "u":12.75, "v":231.53, "w":7.10}, 
                lsr={"u_lsr":0.0, "v_lsr":230.6, "w_lsr":0.0},
                name_for_output="gums_mod", 
                model_name="GUMS_dr3_mod_dens", max_distance=25,
                extinction_map_kwargs={"name":"gums"}, 
                evolution_class={"name":"MIST", "interpolator":"CharonInterpolator"},
                chosen_bands=["Gaia_G_EDR3", "Gaia_BP_EDR3", "Gaia_RP_EDR3"],
                maglim=["Gaia_G_EDR3", 21, "keep"], lost_mass_option=2,
                post_processing_kwargs={"name": "ProcessDarkCompactObjects"},
                output_location="/u/mhuston/code/synthpop/outputfiles/gums_comp",
                output_file_type="hdf5", overwrite=True, chunk_size=250000)

In [ ]:
mod1.init_populations()
mod2.init_populations()

In [ ]:
# Catalogs only need generated once
for i,loc in enumerate(locs):
    mod1.process_location(l_deg=loc[0],b_deg=loc[1], solid_angle=solangs[i].value, solid_angle_unit="deg^2")
    mod2.process_location(l_deg=loc[0],b_deg=loc[1], solid_angle=solangs[i].value, solid_angle_unit="deg^2")

### Gaia and GUMS queries - don't rerun if not needed

In [ ]:
comps = ['bulge','disk']
for i,loc in enumerate(locs):
    coord = SkyCoord(l=loc[0] * u.degree, b=loc[1] * u.degree, frame=Galactic)
    print('Querying Gaia')
    job = Gaia.cone_search_async(coord, radius=radii[i])
    gaia_tab = job.get_results().to_pandas()[['ra','dec','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','bp_rp',
                                              "classprob_dsc_combmod_star"]]
    print('Querying GUMS')
    job = GaiaModel.cone_search_async(coord, radius=radii[i])
    gums_tab = job.get_results().to_pandas()[['ra','dec','mag_g','mag_bp','mag_rp','mass','barycentric_distance','population','age','ag',
                                             'feh', 'pmra','pmdec','radial_velocity']]
    print('Saving to files')
    gaia_tab[gaia_tab["classprob_dsc_combmod_star"]>0.5].to_hdf('gums_compare/gaia_'+comps[i]+'.h5',
                                                                key='data',index=False, mode='w')
    gums_tab.to_hdf('gums_compare/gums_'+comps[i]+'.h5',key='data',index=False, mode='w')

### Load saved SynthPop, Gaia, and GUMS catalogs
For SynthPop, we use 1 and 2 to denote model versions (i.e. density modification). The letters b and d to represent bulge and disk fields, respectively.

In [ ]:
cat1b=pandas.read_hdf("/u/mhuston/code/synthpop/outputfiles/gums_comp_v2/GUMS_dr3_l2.000_b-2.000.h5")
cat2b=pandas.read_hdf("/u/mhuston/code/synthpop/outputfiles/gums_comp_v2/GUMS_dr3_mod_dens_l2.000_b-2.000.h5")

cat1d=pandas.read_hdf("/u/mhuston/code/synthpop/outputfiles/gums_comp_v2/GUMS_dr3_l30.000_b30.000.h5")
cat2d=pandas.read_hdf("/u/mhuston/code/synthpop/outputfiles/gums_comp_v2/GUMS_dr3_mod_dens_l30.000_b30.000.h5")

In [ ]:
gaia_tab_b = pandas.read_hdf('gums_compare/gaia_bulge.h5',key='data')
gums_tab_b = pandas.read_hdf('gums_compare/gums_bulge.h5',key='data')

gaia_tab_d = pandas.read_hdf('gums_compare/gaia_disk.h5',key='data')
gums_tab_d = pandas.read_hdf('gums_compare/gums_disk.h5',key='data')

### Plot star counts by population as a function of distance

In [ ]:
fs = (9,6)
xlab,ylab = r'distance (kpc)', r'stellar density (kpc$^{-3}$)'
labs=['GUMS (G<20)','SP (non-norm) (G<20)','SynthPop (G<20)','SynthPop (all stars) ', 'gaia']

In [ ]:
bins=np.arange(0,25.01,0.5)
plt.figure(figsize=fs)
plt.title('bulge stars in inner field')
plt.hist(cat2b[(cat2b['pop']>-0.5) & (cat2b['pop']<0.5)].Dist,bins=bins,histtype='step',color='grey',label=labs[3])
plt.hist(gums_tab_b[(gums_tab_b.mag_g<20) & (gums_tab_b.population==4)].barycentric_distance/1000,bins=bins,histtype='step',color='r',label=labs[0])
plt.hist(cat1b[(cat1b.Gaia_G_EDR3<20) & (cat1b['pop']>-0.5) & (cat1b['pop']<0.5)].Dist,bins=bins,histtype='step',color='b',label=labs[1])
plt.hist(cat2b[(cat2b.Gaia_G_EDR3<20) & (cat2b['pop']>-0.5) & (cat2b['pop']<0.5)].Dist,bins=bins,histtype='step',color='g',label=labs[2])
plt.yscale('log')
plt.xlim(0,15)
plt.legend()
plt.ylabel(ylab); plt.xlabel(xlab)
plt.savefig('figs/dens_bulge.pdf')

In [ ]:
bins=np.arange(0,25.01,0.5)
plt.figure(figsize=fs)
plt.title('thin disk stars in inner field')
plt.hist(cat2b[(cat2b['pop']>0.5) & (cat2b['pop']<7.5)].Dist,bins=bins,histtype='step',color='grey',label=labs[3])
plt.hist(gums_tab_b[(gums_tab_b.mag_g<20) & (gums_tab_b.population==1)].barycentric_distance/1000,bins=bins,histtype='step',color='r',label=labs[0])
plt.hist(cat1b[(cat1b.Gaia_G_EDR3<20) & (cat1b['pop']>0.5) & (cat1b['pop']<7.5)].Dist,bins=bins,histtype='step',color='b',label=labs[1])
plt.hist(cat2b[(cat2b.Gaia_G_EDR3<20) & (cat2b['pop']>0.5) & (cat2b['pop']<7.5)].Dist,bins=bins,histtype='step',color='g',label=labs[2])
plt.yscale('log')
plt.xlim(0,25)
plt.legend(loc='upper right')
plt.ylabel(ylab); plt.xlabel(xlab)
plt.savefig('figs/dens_thin.pdf')

In [ ]:
bins=np.arange(0,25.01,0.5)
plt.figure(figsize=fs)
plt.title('halo stars in outer field')
plt.hist(cat2d[(cat2d['pop']>7.5) & (cat2d['pop']<8.5)].Dist,bins=bins,histtype='step',color='grey',label=labs[3])
plt.hist(gums_tab_d[(gums_tab_d.mag_g<20) & (gums_tab_d.population==3)].barycentric_distance/1000,bins=bins,histtype='step',color='r',label=labs[0])
plt.hist(cat1d[(cat1d.Gaia_G_EDR3<20) & (cat1d['pop']>7.5) & (cat1d['pop']<8.5)].Dist,bins=bins,histtype='step',color='b',label=labs[1])
plt.hist(cat2d[(cat2d.Gaia_G_EDR3<20) & (cat2d['pop']>7.5) & (cat2d['pop']<8.5)].Dist,bins=bins,histtype='step',color='g',label=labs[2])
plt.yscale('log')
plt.xlim(0,25)
plt.legend(loc='upper right')
plt.ylabel(ylab); plt.xlabel(xlab)
plt.savefig('figs/dens_halo.pdf')

In [ ]:
bins=np.arange(0,25.01,0.5)
plt.figure(figsize=fs)
plt.title('thick disk stars in outer field')
plt.hist(cat2d[(cat2d['pop']>8.5) & (cat2d['pop']<10.5)].Dist,bins=bins,histtype='step',color='grey',label=labs[3])
plt.hist(gums_tab_d[(gums_tab_d.mag_g<20) & (gums_tab_d.population==2)].barycentric_distance/1000,bins=bins,histtype='step',color='r',label=labs[0])
plt.hist(cat1d[(cat1d.Gaia_G_EDR3<20) & (cat1d['pop']>8.5) & (cat1d['pop']<10.5)].Dist,bins=bins,histtype='step',color='b',label=labs[1])
plt.hist(cat2d[(cat2d.Gaia_G_EDR3<20) & (cat2d['pop']>8.5) & (cat2d['pop']<10.5)].Dist,bins=bins,histtype='step',color='g',label=labs[2])
plt.yscale('log')
plt.xlim(0,25)
plt.legend()
plt.ylabel(ylab); plt.xlabel(xlab)
plt.savefig('figs/dens_thick.pdf')

### Plot luminosity functions

In [ ]:
bins = np.arange(10,21.3,0.25)
gums_ct,_ = np.histogram(gums_tab_d.mag_g,bins=bins)
gaia_ct,_ = np.histogram(gaia_tab_d.phot_g_mean_mag, bins=bins)
sp0_ct,_ = np.histogram(cat1d.Gaia_G_EDR3, bins=bins)
sp1_ct,_ = np.histogram(cat2d.Gaia_G_EDR3, bins=bins)
x = bins[:-1]
labs=['GUMS','','SynthPop','','Gaia']

fig, axs = plt.subplots(nrows=2,ncols=1,gridspec_kw=dict(height_ratios=[2.5,1],hspace=0.05), figsize=(8,6))
axs[0].step(x, gaia_ct,label=labs[4], color='k',where='post')
axs[0].step(x, gums_ct,label=labs[0], color='r',where='post')
#axs[0].step(x, sp0_ct,label=labs[1], color='b',where='post')
axs[0].step(x, sp1_ct,label=labs[2], color='g',where='post')
axs[0].legend()

axs[1].axhline(1, color='gray')
axs[1].step(x, sp1_ct/gums_ct,where='post',color='r',linestyle='--', label=r"N$_{\rm SynthPop}$/N$_{\rm GUMS}$")
axs[1].step(x, sp1_ct/gaia_ct,where='post',color='k',linestyle='--', label=r"N$_{\rm SynthPop}$/N$_{\rm Gaia}$")
axs[1].set_ylim(0,2.5); axs[1].set_yticks([0,1,2])
axs[1].legend(loc=(0.6,0.95),framealpha=0.95)

axs[0].set_yscale('log')
axs[0].set_title('Outer field')
axs[0].set_xlim(10,21); axs[1].set_xlim(10,21); axs[0].set_xticks([])
axs[1].set_xlabel(r'm$_G$')
axs[0].set_ylabel('N stars')
fig.savefig('figs/lf_outer.pdf')

In [ ]:
bins = np.arange(13,21.3,0.25)
gums_ct,_ = np.histogram(gums_tab_b.mag_g,bins=bins)
gaia_ct,_ = np.histogram(gaia_tab_b.phot_g_mean_mag, bins=bins)
sp0_ct,_ = np.histogram(cat1b.Gaia_G_EDR3, bins=bins)
sp1_ct,_ = np.histogram(cat2b.Gaia_G_EDR3, bins=bins)
x = bins[:-1]
labs=['GUMS','','SynthPop','','Gaia']

fig, axs = plt.subplots(nrows=2,ncols=1,gridspec_kw=dict(height_ratios=[2.5,1],hspace=0.05), figsize=(8,6))
axs[0].step(x, gaia_ct,label=labs[4], color='k',where='post')
axs[0].step(x, gums_ct,label=labs[0], color='r',where='post')
#axs[0].step(x, sp0_ct,label=labs[1], color='b',where='post')
axs[0].step(x, sp1_ct,label=labs[2], color='g',where='post')
axs[0].legend()

axs[1].axhline(1, color='gray')
axs[1].step(x, (sp1_ct/gums_ct),where='post',color='r',linestyle='--', label=r"N$_{\rm SynthPop}$/N$_{\rm GUMS}$")
axs[1].step(x, (sp1_ct/gaia_ct),where='post',color='k',linestyle='--', label=r"N$_{\rm SynthPop}$/N$_{\rm Gaia}$")
axs[1].set_ylim(0,5); axs[1].set_yticks([0,2,4])
axs[1].legend(loc=(0.6,0.95),framealpha=0.95)

axs[0].set_yscale('log')
axs[0].set_title('Inner field')
axs[0].set_xlim(13,21); axs[1].set_xlim(13,21); axs[0].set_xticks([])
axs[1].set_xlabel(r'm$_G$')
axs[0].set_ylabel('N stars')
fig.savefig('figs/lf_inner.pdf')

### Color-magnitude diagrams

In [ ]:
extlaw = mod1.populations[0].extinction
AG_A0 = extlaw.Alambda_Aref(0.673)/extlaw.Alambda_Aref(0.55)
ABP_A0 = extlaw.Alambda_Aref(0.532)/extlaw.Alambda_Aref(0.55)
ARP_A0 = extlaw.Alambda_Aref(0.797)/extlaw.Alambda_Aref(0.55)
ABP_AG = ABP_A0/AG_A0
ARP_AG = ARP_A0/AG_A0

In [ ]:
fig, axs = plt.subplots(nrows=2,ncols=3,figsize=(15,10),sharey='row',sharex=True)
samp=4
cat2d_s = cat2d[::samp]
gums_tab_s = gums_tab_d[::samp]
gaia_tab_s = gaia_tab_d[::samp]
axs[0][0].plot(cat2d_s.Gaia_BP_EDR3-cat2d_s.Gaia_RP_EDR3, cat2d_s.Gaia_G_EDR3, 'k,')
axs[0][1].plot(gums_tab_s.mag_bp-gums_tab_s.mag_rp, gums_tab_s.mag_g,'k,')
axs[0][2].plot(gaia_tab_s.phot_bp_mean_mag-gaia_tab_s.phot_rp_mean_mag, gaia_tab_s.phot_g_mean_mag,'k,')
fig.delaxes(axs[1][2])

axs[1][0].plot(cat2d_s.Gaia_BP_EDR3-cat2d_s.Gaia_RP_EDR3 - (cat2d_s.A0*ABP_A0 - cat2d_s.A0*ARP_A0), 
                cat2d_s.Gaia_G_EDR3 - 5*np.log10(cat2d_s.Dist*100) - cat2d_s.A0*AG_A0, 'k,')
axs[1][1].plot(gums_tab_s.mag_bp-gums_tab_s.mag_rp - (gums_tab_s.ag*ABP_AG-gums_tab_s.ag*ARP_AG), 
               gums_tab_s.mag_g - 5*np.log10(gums_tab_s.barycentric_distance/10) - gums_tab_s.ag,'k,')

axs[0][0].set_ylim(21,11.5); axs[0][0].set_xlim(0,3); axs[1][0].set_ylim(12.5,-1)
axs[0][0].set_ylabel(r'm$_G$')
axs[1][0].set_ylabel(r'M$_G$')
for ax in axs[0]: ax.set_xlabel(r'm$_{BP}$ - m$_{RP}$')
for ax in axs[1]: ax.set_xlabel(r'M$_{BP}$ - M$_{RP}$')
axs[0][0].set_title('SynthPop'); axs[0][1].set_title('Gaia Universe Model'); axs[0][2].set_title('Gaia DR3')
axs[1][0].set_xticks([0,1,2,3])
plt.tight_layout()
plt.savefig('figs/cmd_compare.pdf')

### A quick look at additional initial stellar properties

In [ ]:
# The ages of the GUMS stars are discretized
np.unique(gums_tab_d.age)

In [ ]:
bins=np.arange(0,25.01,0.5)
plt.figure(figsize=fs)
plt.title('ages')
#plt.hist(cat2d[(cat2d['pop']>8.5) & (cat2d['pop']<10.5)].Dist,bins=bins,histtype='step',color='grey',label=labs[3])
plt.hist(gums_tab_d[(gums_tab_d.mag_g<20)].age,bins=bins,histtype='step',color='r',label=labs[0])
#plt.hist(cat1d[(cat1d.Gaia_G_EDR3<20) & (cat1d['pop']>8.5) & (cat1d['pop']<10.5)].age,bins=bins,histtype='step',color='b',label=labs[1])
plt.hist(cat2d[(cat2d.Gaia_G_EDR3<20)].age,bins=bins,histtype='step',color='g',label=labs[2])
plt.yscale('log')
#plt.xlim(0,25)
plt.legend()
plt.savefig('figs/ages.pdf')

In [ ]:
bins=np.arange(-4,1.01,0.25)
plt.figure(figsize=fs)
plt.title('metallicity')
#plt.hist(cat2d[(cat2d['pop']>8.5) & (cat2d['pop']<10.5)].Dist,bins=bins,histtype='step',color='grey',label=labs[3])
plt.hist(gums_tab_d[(gums_tab_d.mag_g<20)].feh,bins=bins,histtype='step',color='r',label=labs[0])
#plt.hist(cat1d[(cat1d.Gaia_G_EDR3<20) & (cat1d['pop']>8.5) & (cat1d['pop']<10.5)].age,bins=bins,histtype='step',color='b',label=labs[1])
plt.hist(cat2d[(cat2d.Gaia_G_EDR3<20)]['Fe/H_evolved'],bins=bins,histtype='step',color='g',label=labs[2])
plt.hist(cat2d[(cat2d.Gaia_G_EDR3<20)]['Fe/H_initial'],bins=bins,histtype='step',color='b',label=labs[2])
plt.yscale('log')
plt.xlim(-4,1)
plt.legend()
plt.savefig('figs/metallicies.pdf')